In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from category_encoders import LeaveOneOutEncoder
from sklearn.impute import SimpleImputer

import joblib
import pickle
car_instance = pd.DataFrame({
    'manufacturer': ['LEXUS'],
    'model': ['GX 460'],
    'prod._year': [2010],
    'category': ['Jeep'],
    'leather_interior': ['Yes'],
    'fuel_type': ['Petrol'],
    'engine_volume': ['4.6'],
    'mileage': ['275240 km'],
    'cylinders': [8.0],
    'gear_box_type': ['Automatic'],
    'drive_wheels': ['4x4'],
    'doors': ['4-May'],
    'wheel': ['Left wheel'],
    'color': ['Silver'],
    'airbags':[0.0]
})

car_instance2 = pd.DataFrame({
    'manufacturer': ['BMW'],
    'model': ['535'],
    'prod._year': [2013],
    'category': ['Sedan'],
    'leather_interior': ['Yes'],
    'fuel_type': ['Hybrid'],
    'engine_volume': ['3'],
    'mileage': ['151586 km'],
    'cylinders': [6.0],
    'gear_box_type': ['Automatic'],
    'drive_wheels': ['Rear'],
    'doors': ['4-May'],
    'wheel': ['Left wheel'],
    'color': ['White'],
    'airbags':[12.0]
})

# Preprocess the specific car instance
car_instance.columns = car_instance.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
car_instance['leather_interior'] = car_instance['leather_interior'].map({'Yes': 1, 'No': 0})
car_instance['mileage'] = car_instance['mileage'].str.replace(' km', '').astype(int)

car_instance['engine_volume'] = car_instance['engine_volume'].astype(str)
car_instance['is_turbo'] = car_instance['engine_volume'].apply(lambda x: 1 if 'Turbo' in x else 0)
car_instance['engine_volume'] = car_instance['engine_volume'].str.replace(' Turbo', '').astype(float)

# Apply logarithmic transformation to the car instance
car_instance['prod._year'] = np.log(car_instance['prod._year'] + 1)
car_instance['mileage'] = np.log(car_instance['mileage'] + 1)

# Temporarily add the 'price' column to align with the encoder's expected input dimensions
car_instance['price'] =0

# Preprocess the specific car instance
car_instance2.columns = car_instance2.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
car_instance2['leather_interior'] = car_instance2['leather_interior'].map({'Yes': 1, 'No': 0})
car_instance2['mileage'] = car_instance2['mileage'].str.replace(' km', '').astype(int)

car_instance2['engine_volume'] = car_instance2['engine_volume'].astype(str)
car_instance2['is_turbo'] = car_instance2['engine_volume'].apply(lambda x: 1 if 'Turbo' in x else 0)
car_instance2['engine_volume'] = car_instance2['engine_volume'].str.replace(' Turbo', '').astype(float)

# Apply logarithmic transformation to the car instance
car_instance2['prod._year'] = np.log(car_instance2['prod._year'] + 1)
car_instance2['mileage'] = np.log(car_instance2['mileage'] + 1)

# Temporarily add the 'price' column to align with the encoder's expected input dimensions
car_instance2['price'] =0



# Load data
data = pd.read_csv("E:/Licenta/AIModel/AIModel/car_price_prediction.csv")

# Check for duplicate rows and remove them
data = data.drop_duplicates()

# Transform column names to snake case
data.columns = data.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')

# Process 'engine_volume' and create 'is_turbo' feature
data['engine_volume'] = data['engine_volume'].astype(str)
data['is_turbo'] = data['engine_volume'].apply(lambda x: 1 if 'Turbo' in x else 0)
data['engine_volume'] = data['engine_volume'].str.replace(' Turbo', '').astype(float)

# Drop unnecessary columns
data = data.drop(['id'], axis=1)
data = data.drop(['levy'], axis=1)

# Handle NaN and infinity values
data.replace([np.inf, -np.inf,'-'], np.nan, inplace=True)

# Remove rows with NaN values
data = data.dropna(axis=1)

# Process 'mileage'
data['mileage'] = data['mileage'].str.replace(' km', '').astype(int)

# Map 'leather_interior' to binary values
data['leather_interior'] = data['leather_interior'].map({'Yes': 1, 'No': 0})

# Ensure all values in 'prod._year' and 'mileage' are positive
data = data[(data['prod._year'] > 0) & (data['mileage'] > 0)]

data['prod._year'] = np.log(data['prod._year'] + 1)
data['mileage'] = np.log(data['mileage'] + 1)

# Encode categorical features
categorical_columns = ['manufacturer', 'model', 'category', 'fuel_type', 'gear_box_type', 'drive_wheels', 'doors', 'wheel', 'color']
encoder = LeaveOneOutEncoder(cols=categorical_columns)
data = encoder.fit_transform(data, data['price'])
car_instance_encoded=encoder.transform(car_instance)
car_instance_encoded2=encoder.transform(car_instance2)

# Split data into features and target variable
x = data.drop(['price'], axis=1)
y = data['price']

car_instance_encoded = car_instance_encoded.drop(columns=['price'])
car_instance_encoded2 = car_instance_encoded2.drop(columns=['price'])

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Check for NaN values
# print("NaN values in x_train:\n", x_train.isnull().sum())
# print("NaN values in x_test:\n", x_test.isnull().sum())

# Impute missing values (if any) in x_train and x_test
imputer = SimpleImputer(strategy='mean')
x_train_s = imputer.fit_transform(x_train)
x_test_s = imputer.transform(x_test)
car_instance_encoded_imputer=imputer.transform(car_instance_encoded)
car_instance_encoded_imputer2=imputer.transform(car_instance_encoded2)

# Define models
reg = LinearRegression()
forest = RandomForestRegressor(n_estimators=150)
bagging = BaggingRegressor()

models = {'RandomForest': forest, 'Bagging': bagging, 'LinearRegression': reg}

# Results dictionary
results = {'Predicted price 2': [],'Model': [],'Predicted Price':[], 'MAE (Train)': [], 'MAE (Test)': [], 'R-squared (Train)': [], 'R-squared (Test)': []}

# Loop through models
for model_name, model in models.items():
    # Train the model on the scaled training data
    model.fit(x_train_s, y_train)

    # Save the trained model
    # with open(f'{model_name}_model.pkl', 'wb') as file:
    #     pickle.dump(model, file)

    # Predictions on the scaled training set
    y_train_pred = model.predict(x_train_s)

    # Predictions on the scaled evaluation set
    y_test_pred = model.predict(x_test_s)

    predicted_price=model.predict(car_instance_encoded_imputer)
    predicted_price2=model.predict(car_instance_encoded_imputer2)
    predicted_price3 = model.predict([x_train_s[0]])
    
    # Calculate MAE and R-squared for training and evaluation sets
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)

    # Append results to the dictionary
    results['Model'].append(model_name)
    results['Predicted Price'].append(predicted_price)
    results['Predicted price 2'].append(predicted_price2)

    results['MAE (Train)'].append(mae_train)
    results['MAE (Test)'].append(mae_test)
    results['R-squared (Train)'].append(round(r2_train, 3))
    results['R-squared (Test)'].append(round(r2_test, 3))

    # Print the first 10 predicted price vs actual price for each row in the test set
    print(f"\n{model_name} Model Predictions:")
    print("Predicted Price vs Actual Price (First 10)")
    for i, (actual, predicted) in enumerate(zip(y_train, y_train_pred)):
        print(f"Actual: {actual}, Predicted: {predicted}")
        if i == 9:  # Stop after printing 10 pairs
            break

# Save the encoder and training columns
joblib.dump(encoder, 'encoder.joblib')
joblib.dump(x_train.columns, 'train_columns.joblib')

# Create DataFrame from results dictionary
results_df = pd.DataFrame(results)

# Display the results
print(results_df)



C:\Users\arbac\anaconda3\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)



RandomForest Model Predictions:
Predicted Price vs Actual Price (First 10)
Actual: 7213, Predicted: 7213.0
Actual: 22894, Predicted: 22894.0
Actual: 3500, Predicted: 3500.0
Actual: 39201, Predicted: 39201.0
Actual: 23104, Predicted: 23101.986666666668
Actual: 31500, Predicted: 31513.413333333334
Actual: 20385, Predicted: 20385.0
Actual: 1333, Predicted: 1332.78
Actual: 15367, Predicted: 15367.0
Actual: 18503, Predicted: 18504.373333333333

Bagging Model Predictions:
Predicted Price vs Actual Price (First 10)
Actual: 7213, Predicted: 7213.0
Actual: 22894, Predicted: 22894.0
Actual: 3500, Predicted: 3500.0
Actual: 39201, Predicted: 39201.0
Actual: 23104, Predicted: 23104.6
Actual: 31500, Predicted: 31518.6
Actual: 20385, Predicted: 20385.0
Actual: 1333, Predicted: 1333.0
Actual: 15367, Predicted: 15367.0
Actual: 18503, Predicted: 18503.0

LinearRegression Model Predictions:
Predicted Price vs Actual Price (First 10)
Actual: 7213, Predicted: 2877.839292294346
Actual: 22894, Predicted: 20

In [1]:

import seaborn as sns
import pandas as pd
import numpy as np

# Load data
data2 = pd.read_csv("E:/Licenta/AIModel/AIModel/car_price_prediction.csv")
# Handle NaN and infinity values
data2.replace([np.inf, -np.inf,'-'], np.nan, inplace=True)

data2.dropna(axis='columns');

# Compute correlation matrix
correlation_matrix = data2.corr()

# Filter to include only 'price' column
price_correlation = correlation_matrix[['price']]

# Plot the heatmap
plt.figure(figsize=(10, 12))
sns.heatmap(price_correlation, annot=True, cmap="YlGnBu", yticklabels=correlation_matrix.index)
plt.title('Correlation with Price')
plt.show()


ValueError: could not convert string to float: 'LEXUS'

In [3]:
model.predict([x_train_s[0]])[0]

2877.839292294346

In [5]:
x_train_s[0]

array([1.47907967e+04, 1.30186552e+04, 7.60090246e+00, 2.39606110e+04,
       0.00000000e+00, 1.76833439e+04, 2.00000000e+00, 1.21495076e+01,
       4.00000000e+00, 1.62940651e+04, 1.97024462e+04, 1.74731191e+04,
       9.11767601e+03, 1.41431082e+04, 4.00000000e+00, 0.00000000e+00])

In [3]:
car_instance_encoded_imputer

array([[2.03538215e+04, 2.78303333e+04, 7.60638739e+00, 2.39574057e+04,
        1.00000000e+00, 1.76822471e+04, 4.60000000e+00, 1.25254024e+01,
        8.00000000e+00, 1.62933584e+04, 1.96991690e+04, 1.89276839e+04,
        1.97170678e+04, 1.55133575e+04, 0.00000000e+00, 0.00000000e+00]])

In [9]:
x_train_s

array([[1.47907967e+04, 1.30186552e+04, 7.60090246e+00, ...,
        1.41431082e+04, 4.00000000e+00, 0.00000000e+00],
       [1.45071940e+04, 1.27498197e+04, 7.60738143e+00, ...,
        1.84315536e+04, 8.00000000e+00, 0.00000000e+00],
       [1.86144765e+04, 9.19472727e+03, 7.60240134e+00, ...,
        9.81730464e+03, 1.00000000e+01, 0.00000000e+00],
       ...,
       [1.00746880e+04, 8.33215000e+03, 7.60986220e+00, ...,
        1.41501134e+04, 1.20000000e+01, 0.00000000e+00],
       [1.45136839e+04, 1.27748673e+04, 7.61085279e+00, ...,
        1.90596086e+04, 1.20000000e+01, 0.00000000e+00],
       [1.45136211e+04, 1.08497039e+04, 7.60539236e+00, ...,
        1.90595626e+04, 1.20000000e+01, 0.00000000e+00]])

In [ ]:
car_instance2 = pd.DataFrame({
    'manufacturer': ['LEXUS'],
    'model': ['GX 460'],
    'prod._year': [2010],
    'category': ['Jeep'],
    'leather_interior': ['Yes'],
    'fuel_type': ['Petrol'],
    'engine_volume': ['4.6'],
    'mileage': ['275240 km'],
    'cylinders': [8.0],
    'gear_box_type': ['Automatic'],
    'drive_wheels': ['4x4'],
    'doors': ['4-May'],
    'wheel': ['Left wheel'],
    'color': ['Silver'],
    'airbags':[0.0]
})

# Preprocess the specific car instance
car_instance.columns = car_instance.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
car_instance['leather_interior'] = car_instance['leather_interior'].map({'Yes': 1, 'No': 0})
car_instance['mileage'] = car_instance['mileage'].str.replace(' km', '').astype(int)

car_instance['engine_volume'] = car_instance['engine_volume'].astype(str)
car_instance['is_turbo'] = car_instance['engine_volume'].apply(lambda x: 1 if 'Turbo' in x else 0)
car_instance['engine_volume'] = car_instance['engine_volume'].str.replace(' Turbo', '').astype(float)

# Apply logarithmic transformation to the car instance
car_instance['prod._year'] = np.log(car_instance['prod._year'] + 1)
car_instance['mileage'] = np.log(car_instance['mileage'] + 1)

# Temporarily add the 'price' column to align with the encoder's expected input dimensions
car_instance['price'] =0